In [ ]:
import pandas as pd
import os
import numpy as np
import datetime
import chardet
import time
from matplotlib import pyplot as plt
from collections import Counter
from itertools import combinations

In [ ]:
df = pd.read_csv('results.csv', sep=',', encoding='latin-1', on_bad_lines='skip', low_memory=False)
analysis = ["OR Result", "Confluence Inter Result", "OA Inter Result", "DF Inter Result", "PDG Result"]
result_analysis = ["Confluence Inter", "OA Inter", "left right DFP-Inter", "right left DFP-Inter", "left right PDG", "right left PDG"]

In [ ]:
df = df[df["Confluence Inter"] != "not-found"]
df = df[df["LOI"] != "-"]
print(df["Confluence Inter"].count())



In [ ]:
def get_name_analysis(list_name):
    names = []
    for i in list_name:
        if ("Confluence" in i):
            names.append("Confluence Inter")
        elif ("OA" in i):
            names.append("OA Inter")
        elif ("DF" in i):
            names.append("left right DFP-Inter")
            names.append("right left DFP-Inter")
        elif ("PDG" in i):
            names.append("left right PDG")
            names.append("right left PDG")

    return names

def get_reverse_name(lists):
    names = []
    for elem_list in lists:
        aux_list = []
        for i in elem_list:
            if ("Confluence Inter" in i):
                aux_list.append("CF")
            elif ("OA Inter" in i):
                aux_list.append("OA")
            elif ("left right DFP-Inter" in i):
                aux_list.append("DF")
            elif ("left right PDG" in i):
                aux_list.append("PDG")
        names.append(aux_list)
    return names


In [ ]:
# Lista dos elementos
elements = ["Confluence", "OA", "DF", "PDG"]
combinations_list = []
# Gerar todas as combinações possíveis de 2 a 4 elementos sem repetições
for length in range(1, len(elements) + 1):
    for combination in combinations(elements, length):
        combinations_list.append(list(combination))

print(combinations_list)

# gerando todas as combinações possíveis

analysis_combination = []
for i in combinations_list:
    analysis_combination.append(get_name_analysis(i))
print(analysis_combination)

In [ ]:
def calculate_matrix(columns):
    results = []
    for index, row in df.iterrows():
        values = [row[column] for column in columns]
        loi = row["LOI"]
        or_value = any(value != 'FALSE' for value in values)
        result = ""
        if or_value == True and loi == 'Yes':
            result = "TRUE POSITIVE"
        elif or_value == False and loi == 'No':
            result = "TRUE NEGATIVE"
        elif or_value == False and loi == 'Yes':
            result = "FALSE NEGATIVE"
        elif or_value == True and loi == 'No':
            result = "FALSE POSITIVE"
        
        results.append(result)
    return results

In [ ]:
# verificando se o OR da planilha é igual ao da função

r = calculate_matrix(result_analysis)
# print(r)

lista = [i for i in df["OR Result"]]

if r == lista:
    print("Iguais")

# print(lista)

In [ ]:
#Algoritmo
#Se todos os TP e TN de uma análise A1 são também TP e TN de outra análise A2, e A2 tem pelo 
#menos um TP ou TN a mais que A1, o algoritmo indica que a gente deve ficar apenas com A2

#Se todos os TP e TN de uma análise A1 são também TP e TN de outra análise A2, e A2 não tem 
#nenhum TP ou TN a mais que A1, o algoritmo indica que a gente deve ficar com a mais rápida entre A1 e A2

#Se nem todos os TP e TN de uma análise A1 são também TP e TN de outra análise A2, e vice-versa, 
#o algoritmo sugere a gente ficar com o OR de A1 e A2

#Se todos os TP e TN de uma análise (CONJUNTO - OU de n análise) são também TP e TN de outra análise A2, e A2 tem pelo 
#menos um TP ou TN a mais que A1, o algoritmo indica que a gente deve ficar apenas com A2

def compare_algorithm(list1, list2):
    TPIgual = True
    TNIgual = True
    for (l1, l2) in zip(list1, list2):
        if (l1 == "TRUE POSITIVE" and l1 != l2):
            TPIgual = False
        if (l1 == "TRUE NEGATIVE" and l1 != l2):
            TNIgual = False


    if (TPIgual and TNIgual and (list2.count("TRUE POSITIVE") > list1.count("TRUE POSITIVE") or list2.count("TRUE NEGATIVE") > list1.count("TRUE NEGATIVE"))):
        print("Vamos optar por Last!")
    elif (TPIgual and TNIgual and (list2.count("TRUE POSITIVE") == list1.count("TRUE POSITIVE") or list2.count("TRUE NEGATIVE") == list1.count("TRUE NEGATIVE"))):
        print("Ficar com a mais rápida!")
    else:
        print("Vamos optar pelo First!")

In [ ]:
def count_fp_fn(list_result):
    # Criar um contador dos elementos da lista
    element_count = Counter(list_result)

    result = []
    # Imprimir a contagem de elementos repetidos
    for element, count in element_count.items():
        if count > 1:
            result.append((str(element)+": "+str(count)))
    return result

In [ ]:
class Longest:
    maiorPrecision = -1.0
    maiorRecall = -1.0
    maiorF1 = -1.0
    maiorAcuracia = -1.0
    mPrecision = []
    mRecall = []
    mF1 = []
    mAcuracia = []
    mPrecision_t = []
    mRecall_t = []
    mF1_t = []
    mAcuracia_t = []

    def __init__(self):
        self.maiorPrecision = -1.0
        self.maiorRecall = -1.0
        self.maiorF1 = -1.0
        self.maiorAcuracia = -1.0
        mPrecision = []
        mRecall = []
        mF1 = []
        mAcuracia = []
        mPrecision_t = []
        mRecall_t = []
        mF1_t = []
        mAcuracia_t = []


    def confusion_matrix(self, options, values_elem):

        # Inicializar as variáveis
        tp = None
        fp = None
        tn = None
        fn = None
        
        # Extrair os valores dos elementos da lista
        for option in options:
            if "TRUE POSITIVE" in option:
                tp = int(option.split(': ')[1])
            elif "FALSE POSITIVE" in option:
                fp = int(option.split(': ')[1])
            elif "TRUE NEGATIVE" in option:
                tn = int(option.split(': ')[1])
            elif "FALSE NEGATIVE" in option:
                fn = int(option.split(': ')[1])

        # Calcular as métricas se todos os valores foram extraídos
        if tp is not None and fp is not None and tn is not None and fn is not None:
            precision = tp / (tp + fp)
            recall = tp / (tp + fn)
            f1_score = 2 * (precision * recall) / (precision + recall)
            accuracy = (tp + tn) / (tp + tn + fp + fn)

            if (precision > self.maiorPrecision):
                self.maiorPrecision = precision
                self.mPrecision = []
                self.mPrecision.append(values_elem)

            if (precision == self.maiorPrecision):
                if values_elem not in self.mPrecision:
                    self.mPrecision.append(values_elem)
                
            if (recall > self.maiorRecall):
                self.maiorRecall = recall
                self.mRecall = []
                self.mRecall.append(values_elem)

            if (recall == self.maiorRecall):
                if values_elem not in self.mRecall:
                    self.mRecall.append(values_elem)

                
            if (f1_score > self.maiorF1):
                self.maiorF1 = f1_score
                self.mF1 = []
                self.mF1.append(values_elem)

            if (f1_score == self.maiorF1):
                if values_elem not in self.mF1:
                    self.mF1.append(values_elem)
            
            if (accuracy > self.maiorAcuracia):            
                self.maiorAcuracia = accuracy
                self.mAcuracia = []
                self.mAcuracia.append(values_elem)

            if (accuracy == self.maiorAcuracia):
                if values_elem not in self.mAcuracia:
                    self.mAcuracia.append(values_elem)

            # Imprimir as métricas
            print(f"Precision: {precision:.2f}")
            print(f"Recall: {recall:.2f}")
            print(f"F1 Score: {f1_score:.2f}")
            print(f"Accuracy: {accuracy:.2f}")
                
            result_metrics = {
                "precision": precision,
                "recall": recall,
                "f1_score": f1_score,
                "accuracy": accuracy
            }
            return result_metrics
        else:
            print("Não foi possível extrair todos os valores necessários.")

        return None


In [ ]:
#Escolhendo qual o melhor resultado com base no Algoritmo de comparação
best = Longest()

for first in analysis_combination:
    for last in analysis_combination:
        if first != last:  
            print(first, last)      
            r_first = calculate_matrix(first)
            r_last = calculate_matrix(last)

            print("Comparing OR of the first", first, "with the last", last)
            print("First:", count_fp_fn(r_first))
            best.confusion_matrix(count_fp_fn(r_first), first)
            print("Last: ", count_fp_fn(r_last))
            best.confusion_matrix(count_fp_fn(r_last), last)
            compare_algorithm(r_first, r_last)
            print()

print(f"Precision: {best.maiorPrecision:.2f}", best.mPrecision)
print(f"Recall: {best.maiorRecall:.2f}", best.mRecall)
print(f"F1-score: {best.maiorF1:.2f}", best.mF1)
print(f"Accuracy: {best.maiorAcuracia:.2f}", best.mAcuracia)


In [ ]:
data = {
    'Metric': ['Precision', 'Recall', 'F1-score', 'Accuracy'],
    'Value': [round(best.maiorPrecision, 2), round(best.maiorRecall, 2), round(best.maiorF1, 2), round(best.maiorAcuracia, 2)],
   'Analyses': [get_reverse_name(best.mPrecision), get_reverse_name(best.mRecall), get_reverse_name(best.mF1), get_reverse_name(best.mAcuracia)]
}
dframe = pd.DataFrame(data)

fig, ax = plt.subplots(figsize=(12, 4))
ax.axis('tight')
ax.axis('off')

table = ax.table(cellText=dframe.values, colLabels=dframe.columns, cellLoc='center', loc='center')
table.auto_set_font_size(False)
table.set_fontsize(12)
table.scale(1.2, 1.2)

for i, col in enumerate(dframe.columns):
    col_width = max([len(str(val)) for val in dframe[col]])
    table.auto_set_column_width(i)
    table.auto_set_column_width(col_width)

plt.title("Result of the best combinations", y=0.8)

plt.savefig('best_combinations.jpg', dpi=300, bbox_inches='tight', pad_inches=0.5)



In [ ]:
df = df[df["Confluence Inter"] != "not-found"]
df = df[df["LOI"] != "-"]
print(df["Confluence Inter"].count())

In [ ]:

def check_exclusive(op1, op2, val):
    
    count = 0
    for i, j in zip(op1, op2):
        if (val in i and (val not in j)):
            count = count + 1
    return count

In [ ]:
def check_algorithm_tp_fp(first, last):
    first_tp = check_exclusive(first, last, "TRUE POSITIVE")
    first_fp = check_exclusive(first, last, "FALSE POSITIVE")
    r_first = first_tp-first_fp
    print("TP exclusive first:", first_tp, ", FP exclusive first:", first_fp, ", saldo:", r_first)
    
    last_tp = check_exclusive(last, first, "TRUE POSITIVE")
    last_fp = check_exclusive(last, first, "FALSE POSITIVE")
    r_last = last_tp-last_fp
    print("TP exclusive last:", last_tp, ", FP exclusive last:", last_fp, ", saldo:", r_last)

    if (r_first > r_last):
        print("Escolher first (OR)!")
    elif (r_first < r_last):
        print("Escolher last!")
    else:
        print("Ficar com a mais rápida!")

In [ ]:
#Escolhendo qual o melhor resultado com base na tabela de TP - FP
obj = Longest()
for first_elements in analysis_combination: 
    for last_element in analysis_combination:
        if first_elements != last_element:
            # print(i, first_elements, last_element)
            r_first = calculate_matrix(first_elements)
            r_last = calculate_matrix(last_element)

            print("Comparing OR of the first", first_elements, "with the last", last_element)
            print("First:", count_fp_fn(r_first))
            
            obj.confusion_matrix(count_fp_fn(r_first), first_elements)
            print("Last: ", count_fp_fn(r_last))
            obj.confusion_matrix(count_fp_fn(r_last), last_element)

            check_algorithm_tp_fp(r_first, r_last)
            print()

In [ ]:
# Calcular pontuação para cada resultado
def calculate_score(metrics, weights):
    return sum(metrics[metric] * weights[metric] for metric in metrics)

def check_algorithm_weights(first, last, precision, recall, f1_score, accuracy):
    obj = Longest()
    result_1_metrics = obj.confusion_matrix(count_fp_fn(first), first)
    result_2_metrics = obj.confusion_matrix(count_fp_fn(last), last)
    
    # Definir pesos para as métricas (você pode ajustar esses pesos)
    weights = {
        "precision": precision,
        "recall": recall,
        "f1_score": f1_score,
        "accuracy": accuracy
    }

    score_1 = calculate_score(result_1_metrics, weights)
    score_2 = calculate_score(result_2_metrics, weights)

    print(f"Score First: {score_1:.2f}")
    print(f"Score Last: {score_2:.2f}")

    # Determinar qual resultado é melhor com base na pontuação
    if score_1 > score_2:
        best_result = "First"
    else:
        best_result = "Last"

    print(f"O melhor resultado é {best_result}!")


In [ ]:
#Escolhendo qual o melhor resultado com base na tabela de TP - FP

for first_elements in analysis_combination:
    for last_element in analysis_combination:
        if first_elements != last_element:
                    
            r_first = calculate_matrix(first_elements)
            r_last = calculate_matrix(last_element)

            print("Comparing OR of the first", first_elements, "with the last", last_element)
            print("First:", count_fp_fn(r_first))
            print("Last: ", count_fp_fn(r_last))

            # "precision": 0.4
            # "recall": 0.3
            # "f1_score": 0.2
            # "accuracy": 0.1
            check_algorithm_weights(r_first, r_last, 0.4, 0.3, 0.2, 0.1)


            print()

In [ ]:
#Cálculo de TP e FP exclusivo para cada análise

def check_exclusivo(op1, op2, op3, op4, val):
    if (val in op1 and (val not in op2 and val not in op3 and val not in op4)):
        return 1
    return 0

analysis = ["OR Result", "Confluence Inter Result", "OA Inter Result", "DF Inter Result", "PDG Result"]

cf_tp, df_tp, oa_tp, pdg_tp = 0, 0, 0, 0

cf_fp, df_fp, oa_fp, pdg_fp = 0, 0, 0, 0

for (cf, df, oa, pdg) in zip(df["Confluence Inter Result"], df["DF Inter Result"], df["OA Inter Result"], df["PDG Result"]):
    if ("nan" not in str(cf)):

        cf_tp = cf_tp + check_exclusivo(cf, df, oa, pdg, "TRUE POSITIVE")
        df_tp = df_tp + check_exclusivo(df, cf, oa, pdg, "TRUE POSITIVE")
        oa_tp = oa_tp + check_exclusivo(oa, df, cf, pdg, "TRUE POSITIVE")
        pdg_tp = pdg_tp + check_exclusivo(pdg, cf, df, oa, "TRUE POSITIVE")

        cf_fp = cf_fp + check_exclusivo(cf, df, oa, pdg, "FALSE POSITIVE")
        df_fp = df_fp + check_exclusivo(df, cf, oa, pdg, "FALSE POSITIVE")
        oa_fp = oa_fp + check_exclusivo(oa, df, cf, pdg, "FALSE POSITIVE")
        pdg_fp = pdg_fp + check_exclusivo(pdg, cf, df, oa, "FALSE POSITIVE")

data = {
    'Analysis': ['CF', 'OA', 'DF', 'PDG'],
    'Exclusive TP': [cf_tp, oa_tp, df_tp,  pdg_tp],
    'Exclusive FP': [cf_fp, oa_fp, df_fp,  pdg_fp]
}

dframe = pd.DataFrame(data)
dframe['Balance (TPe-FPe)'] = dframe['Exclusive TP'] - dframe['Exclusive FP']

fig, ax = plt.subplots(figsize=(8, 4))
ax.axis('tight')
ax.axis('off')

table = ax.table(cellText=dframe.values, colLabels=dframe.columns, cellLoc='center', loc='center')
table.auto_set_font_size(False)
table.set_fontsize(12)
table.scale(1.2, 1.2)

plt.title("Exclusive analyses", y=0.8)
plt.savefig('exclusive_analysis.jpg', dpi=300, bbox_inches='tight', pad_inches=0.5)



In [ ]:
# 

df_t = pd.read_csv('resultTime.csv', sep=';', encoding='utf-8', on_bad_lines='skip', low_memory=False)

# analysis = ["OR Result", "Confluence Inter Result", "OA Inter Result", "DF Inter Result", "PDG Result"]
# result_analysis = ["Confluence Inter", "OA Inter", "left right DFP-Inter", "right left DFP-Inter", "left right PDG", "right left PDG"]


In [ ]:
def get_name_analysis_time(list_name, config):
    names = []
    for i in list_name:
        if ("Confluence" in str(i) and "Config CF left-base" not in names and "Confluence left-base" not in names):
            if config:
                names.append("Config CF left-base")
                names.append("Config CF right-base")
            names.append("Confluence left-base")
            names.append("Confluence right-base")
        elif ("OA" in str(i)):
            if config:
                names.append("Config OA")
            names.append("OA")
        elif ("DF" in str(i) and "Config DFP left" not in names and "DFP left" not in names):
            if config:
                names.append("Config DFP left")
                names.append("Config DFP right")
            names.append("DFP left")
            names.append("DFP right")
        elif ("PDG" in str(i) and "Config PDG left" not in names and "PDG left" not in names):
            if config:
                names.append("Config PDG left")
                names.append("Config PDG right")
            names.append("PDG left")
            names.append("PDG right")

    return names

In [ ]:
def sum_n_esimo_element(lista_de_listas, n):
    return sum(sublista[n] for sublista in lista_de_listas)

def get_sum_all_list(all_list):
    return [sum_n_esimo_element(all_list, i) for i in range(len(all_list[0]))]

def get_total_time(list_analysis):
    result = []
    for column_analysis in list_analysis:
        actual_time = [i for i in df_t[column_analysis]]
        result.append(actual_time)
    return result

def get_mean(values):
    return pd.Series(get_sum_all_list(values)).mean()

def get_median(values):
    return pd.Series(get_sum_all_list(values)).median()

def get_sum(values):
    return pd.Series(get_sum_all_list(values)).sum()

def get_standard_desviation(values):
    return pd.Series(get_sum_all_list(values)).std()

In [ ]:
def get_mean_metric(metric, config):
    metric_t = []
    for i in metric:
        aux_list = get_name_analysis_time(i, config)
        result = get_total_time(aux_list)
        value = round(pd.Series(get_mean(result)).sum(), 2)
        metric_t.append(value)
    return metric_t

def get_sum_metric(metric, config):
    metric_t = []
    for i in metric:
        aux_list = get_name_analysis_time(i, config)
        result = get_total_time(aux_list)
        value = round(pd.Series(get_sum(result)).sum(), 2)
        metric_t.append(value)
    return metric_t

def get_median_metric(metric, config):
    metric_t = []
    for i in metric:
        aux_list = get_name_analysis_time(i, config)
        result = get_total_time(aux_list)
        value = round(pd.Series(get_median(result)).sum(), 2)
        metric_t.append(value)
    return metric_t

def get_std_metric(metric, config):
    metric_t = []
    for i in metric:
        aux_list = get_name_analysis_time(i, config)
        result = get_total_time(aux_list)
        value = round(pd.Series(get_standard_desviation(result)).sum(), 2)
        metric_t.append(value)
    return metric_t


In [ ]:

with_config = True

colums = df_t.columns

print("Analyzing", best.mPrecision)
mean_p = get_mean_metric(best.mPrecision, with_config)
median_p = get_median_metric(best.mPrecision, with_config)
# sum_p = get_sum_metric(best.mPrecision, with_config)
std_p = get_std_metric(best.mPrecision, with_config)

# out_precision = f"Mean: {mean_p} Median: {median_p} Sum: {sum_p} Standard: {std_p}"
out_precision = [mean_p, median_p, std_p]

print(out_precision)

print("Analyzing", best.mRecall)
mean_p = get_mean_metric(best.mRecall, with_config)
median_p = get_median_metric(best.mRecall, with_config)
# sum_p = get_sum_metric(best.mRecall, with_config)
std_p = get_std_metric(best.mRecall, with_config)

# out_recall = f"Mean: {mean_p} Median: {median_p} Sum: {sum_p} Standard: {std_p}"
out_recall = [mean_p, median_p, std_p]

print(out_recall)

print("Analyzing", best.mF1)

mean_p = get_mean_metric(best.mF1, with_config)
median_p = get_median_metric(best.mF1, with_config)
# sum_p = get_sum_metric(best.mF1, with_config)
std_p = get_std_metric(best.mF1, with_config)

# out_f1 = f"Mean: {mean_p} Median: {median_p} Sum: {sum_p} Standard: {std_p}"
out_f1 = [mean_p, median_p, std_p]
print(out_f1)

print("Analyzing", best.mAcuracia)
mean_p = get_mean_metric(best.mAcuracia, with_config)
median_p = get_median_metric(best.mAcuracia, with_config)
# sum_p = get_sum_metric(best.mAcuracia, with_config)
std_p = get_std_metric(best.mAcuracia, with_config)

# out_accuracy = f"Mean: {mean_p}\nMedian: {median_p}\nSum: {sum_p}\nStandard: {std_p}\n"
out_accuracy = [mean_p, median_p, std_p]
print(out_accuracy)



In [ ]:
def convert_list_to_tuple(out):
    result = []
    for index in range(len(out[0])):
        val = [x[index] for x in out]
        result.append(tuple(val))
    return result

In [ ]:
data = {
    'Metric': ['Precision', 'Recall', 'F1-score', 'Accuracy'],
    'Value': [round(obj.maiorPrecision, 2), round(obj.maiorRecall, 2), round(obj.maiorF1, 2), round(obj.maiorAcuracia, 2)],
    'Analyses': [get_reverse_name(obj.mPrecision), get_reverse_name(obj.mRecall), get_reverse_name(obj.mF1), get_reverse_name(obj.mAcuracia)],
    'Time (s) (mean, median, standard)': [convert_list_to_tuple(out_precision), convert_list_to_tuple(out_recall), convert_list_to_tuple(out_f1), convert_list_to_tuple(out_accuracy)]
}
dframe = pd.DataFrame(data)

fig, ax = plt.subplots(figsize=(12, 4))
ax.axis('tight')
ax.axis('off')

table = ax.table(cellText=dframe.values, colLabels=dframe.columns, cellLoc='center', loc='center')
table.auto_set_font_size(False)
table.set_fontsize(12)
table.scale(1.2, 1.2)

for i, col in enumerate(dframe.columns):
    col_width = max([len(str(val)) for val in dframe[col]])
    table.auto_set_column_width(i)
    table.auto_set_column_width(col_width)

plt.title("Result of the best combinations", y=0.8)

plt.savefig('best_combinations_time.jpg', dpi=300, bbox_inches='tight', pad_inches=0.5)



In [ ]:
print(len(analysis_combination))
print(analysis_combination)

In [ ]:
soot_results = pd.read_csv('soot-results.csv', sep=';', encoding='latin-1', on_bad_lines='skip', low_memory=False)
loi = pd.read_csv('LOI.csv', sep=';', encoding='latin-1', on_bad_lines='skip', low_memory=False)

In [ ]:
import pandas as pd
import numpy as np
import itertools
from collections import Counter
from matplotlib import pyplot as plt
from fpdf import FPDF

class ReportAnalysis:

    def __init__(self, path_result, path_ground_truth):
        self.soot_results = pd.read_csv(path_result, sep=';', encoding='latin-1', on_bad_lines='skip', low_memory=False)
        self.loi = pd.read_csv(path_ground_truth, sep=';', encoding='latin-1', on_bad_lines='skip', low_memory=False)

        self.generate_results()

    def get_loi(self, project, class_name,  method, merge_commit):

        filter_scenario = (self.loi['Project'] == str(project)) & (self.loi['Merge Commit'] == str(merge_commit)) & (self.loi['Class Name'] == str(class_name)) & (self.loi['Method or field declaration changed by the two merged branches'] == str(method))
        value_LOI = ""

        if filter_scenario.any():
            value_LOI = self.loi.loc[filter_scenario, 'Locally Observable Interference'].values[0]

        return value_LOI

    def calculate_matrix_loi(self, columns):
        results = []
        info_LOI = ['project', 'class', 'method', 'merge commit']

        for index, row in self.soot_results.iterrows():
            list_values = self.soot_results.columns.tolist()
            remove_columns = ['project', 'class', 'method', 'merge commit', 'Time']
            list_values = [coluna for coluna in list_values if coluna not in remove_columns]
            values = [row[column] for column in list_values]

            values_LOI = [row[column] for column in info_LOI]

            loi_actual = self.get_loi(*values_LOI)

            or_value = any(str(value).lower() != 'false' for value in values)

            result = ""
            if or_value == True and loi_actual == 'Yes':
                result = "TRUE POSITIVE"
            elif or_value == False and loi_actual == 'No':
                result = "TRUE NEGATIVE"
            elif or_value == False and loi_actual == 'Yes':
                result = "FALSE NEGATIVE"
            elif or_value == True and loi_actual == 'No':
                result = "FALSE POSITIVE"
            results.append(result)
        return results

    def generate_results(self):

        print("Generating results...")

        FP,TP, FN, TN = 0, 0, 0, 0

        list_columns = self.soot_results.columns.tolist()

        result_matrix = self.calculate_matrix_loi(list_columns)

        for elem, count in Counter(result_matrix).items():
            if (elem == 'FALSE POSITIVE'):
                FP = count
            if (elem == 'FALSE NEGATIVE'):
                FN = count
            if (elem == 'TRUE POSITIVE'):
                TP = count
            if (elem == 'TRUE NEGATIVE'):
                TN = count

        sensitivity = 0 if ((TP + FN) == 0) else (TP / (TP + FN))
        precision = 0 if ((TP + FP) == 0) else (TP / (TP + FP))
        f1_score = 0 if ((2*TP + FP + FN) == 0) else (2*TP / (2*TP + FP + FN))
        accuracy = 0 if ((FP + TP + TN + FN) == 0) else ((TP + TN) / (FP + TP + TN + FN))

        # variable pdf
        pdf = FPDF()

        # add a page
        pdf.add_page()

        # set style and size of font
        # that you want in the pdf
        pdf.set_font("Arial", size = 15)

        # create a cell
        pdf.cell(200, 10, txt = "Results for execution",
                 ln = 1, align = 'C')

        pdf.cell(200, 10, txt = ("Precision: "+str(round(precision, 4))),
                 ln = 2, align = 'L')

        pdf.cell(200, 10, txt = ("Recall: "+str(round(sensitivity, 4))),
                 ln = 2, align = 'L')

        pdf.cell(200, 10, txt = ("F1 Score: "+str(round(f1_score, 4))),
                 ln = 2, align = 'L')

        pdf.cell(200, 10, txt = ("Accuracy: "+str(round(accuracy, 4))),
                 ln = 2, align = 'L')

        pdf.cell(200, 10, txt = ("False Positives: "+str(FP)),
                 ln = 2, align = 'L')

        pdf.cell(200, 10, txt = ("False Negatives: "+str(FN)),
                 ln = 2, align = 'L')

        pdf.cell(200, 10, txt = ("True Positives: "+str(TP)),
                 ln = 2, align = 'L')

        pdf.cell(200, 10, txt = ("True Negatives: "+str(TN)),
                 ln = 2, align = 'L')

        cm = np.array([[TP,  FP], [FN, TN]])
        normalize = False
        target_names = ['Actually Positive', ' Actually Negative']
        target_names2 = ['Predicted Positive', ' Predicted Negative']
        title = "Confusion Matrix"

        cmap = plt.get_cmap('Blues')

        plt.figure(figsize=(8, 6))
        plt.imshow(cm, interpolation='nearest', cmap=cmap)
        plt.title(title, fontsize=16)
        plt.colorbar()

        if target_names is not None:
            tick_marks = np.arange(len(target_names))
            plt.xticks(tick_marks, target_names, fontsize=16)
            plt.yticks(tick_marks, target_names2, fontsize=16)

        if normalize:
            cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

        thresh = cm.max() / 1.5 if normalize else cm.max() / 2
        for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
            if normalize:
                plt.text(j, i, "{:0.4f}".format(cm[i, j]),
                         horizontalalignment="center",
                         color="yellow" if cm[i, j] > thresh else "black", fontsize=23)
            else:
                plt.text(j, i, "{:,}".format(cm[i, j]),
                         horizontalalignment="center",
                         color="yellow" if cm[i, j] > thresh else "black", fontsize=23)
        plt.tight_layout()

        plt.savefig("confusion_matrix.jpg")

        pdf.image("confusion_matrix.jpg", x = None, y = None, w = 160, h = 110, type = 'jpg', link = 'confusion_matrix.jpg')

        # Save the pdf with name .pdf
        pdf.output("output/data/results.pdf")
        # pdf.output("results.pdf")

        print("Results in output/data/results.pdf")

path_ground_truth = "../miningframework/input/LOI.csv"
path_result = '../miningframework/output/data/soot-results.csv'
# path_ground_truth = "LOI.csv"
# path_result = 'soot-results.csv'

print("Reading analyses execution results...")

ReportAnalysis(path_result, path_ground_truth)



In [ ]:
import pandas as pd

def menor_tamanho_chave(dicionario):
    menor_chave = None
    menor_tamanho = float('inf')  # Inicialmente, consideramos o tamanho infinito

    for chave, valor in dicionario.items():
        if isinstance(valor, list) and len(valor) < menor_tamanho:
            menor_chave = chave
            menor_tamanho = len(valor)

    return menor_tamanho
def chaves_com_tamanho_n(dicionario, tamanho):
    chaves = []
    for chave, valor in dicionario.items():
        if isinstance(valor, list) and len(valor) > tamanho:
            chaves.append(chave)
    return chaves

def adjusting_dict(dados_dict):
    # Chave que você deseja dividir
    dicionario = dados_dict
    
    len_dic = menor_tamanho_chave(dados_dict)

    chaves = chaves_com_tamanho_n(dados_dict, len_dic)
    
    for chave in chaves:
        # Verifica se a chave existe no dicionário
        if chave in dicionario:
            # Obtém a lista de valores correspondente à chave
            lista_valores = dicionario[chave]
            
            # Verifica se a lista tem mais de len_dic elementos
            if len(lista_valores) > len_dic:
                
                # Divide a lista em duas partes de len_dic elementos cada
                primeira_parte = lista_valores[:len_dic]
                segunda_parte = lista_valores[len_dic:]
                
                # Cria duas novas chaves no dicionário com as partes divididas
                dicionario[chave + ' left-right'] = primeira_parte
                dicionario[chave + ' right-left'] = segunda_parte
                
                # Exclui a chave antiga
                del dicionario[chave]
            else:
                dicionario[chave] = lista_valores
                

    # Exibe o dicionário atualizado
    return dicionario


def normalize_dict(dic):
    # Encontre o tamanho máximo entre todas as listas
    tamanho_maximo = max(len(valor) for valor in dic.values())
    
    # Percorra as chaves do dicionário
    for chave, valor in dic.items():
        if isinstance(valor, list):
            # Se a lista for menor que o tamanho máximo, preencha com zeros
            while len(valor) < tamanho_maximo:
                valor.append(0)
    
    return dic


dados_dict = {}

arquivo_entrada = 'time.txt'

# Abrir o arquivo de entrada e ler as linhas
with open(arquivo_entrada, 'r') as arquivo:
    linhas = arquivo.readlines()

# Processar as linhas e preencher o dicionário de dados
for linha in linhas:
    partes = linha.strip().split(';')
    if len(partes) == 2:
        coluna, valor = partes[0].strip(), partes[1].strip().replace("s","")        
        if coluna in dados_dict:
            dados_dict[coluna].append(valor)
        else:
            dados_dict[coluna] = [valor]

aux_dic = adjusting_dict(dados_dict)

final_dic = normalize_dict(aux_dic)

# Criar um DataFrame com Pandas
df = pd.DataFrame(final_dic)

# Salvar o DataFrame em uma planilha CSV
df.to_csv('planilha.csv', index=False)



In [ ]:
#!/usr/bin/python

import pandas as pd
import sys

# Check if n was passed as a parameter, otherwise it will default to 10
n = 10
if len(sys.argv) > 1:
    n = int(sys.argv[1])
else:
    n = 10


def menor_tamanho_chave(dicionario):
    menor_chave = None
    menor_tamanho = float('inf')  # Inicialmente, consideramos o tamanho infinito

    for chave, valor in dicionario.items():
        if isinstance(valor, list) and len(valor) < menor_tamanho:
            menor_chave = chave
            menor_tamanho = len(valor)

    return menor_tamanho
def chaves_com_tamanho_n(dicionario, tamanho):
    chaves = []
    for chave, valor in dicionario.items():
        if isinstance(valor, list) and len(valor) > tamanho:
            chaves.append(chave)
    return chaves

def adjusting_dict(dados_dict):
    # Chave que você deseja dividir
    dicionario = dados_dict

    len_dic = menor_tamanho_chave(dados_dict)

    chaves = chaves_com_tamanho_n(dados_dict, len_dic)

    for chave in chaves:
        # Verifica se a chave existe no dicionário
        if chave in dicionario:
            # Obtém a lista de valores correspondente à chave
            lista_valores = dicionario[chave]

            # Verifica se a lista tem mais de len_dic elementos
            if len(lista_valores) > len_dic:

                # Divide a lista em duas partes
                primeira_parte = lista_valores[::2]
                segunda_parte = lista_valores[1::2]

                # Cria duas novas chaves no dicionário com as partes divididas
                dicionario[chave + ' left-right'] = primeira_parte
                dicionario[chave + ' right-left'] = segunda_parte

                # Exclui a chave antiga
                del dicionario[chave]
            else:
                dicionario[chave] = lista_valores


    # Exibe o dicionário atualizado
    return dicionario


def normalize_dict(dic):
    # Encontre o tamanho máximo entre todas as listas
    tamanho_maximo = max(len(valor) for valor in dic.values())

    # Percorra as chaves do dicionário
    for chave, valor in dic.items():
        if isinstance(valor, list):
            # Se a lista for menor que o tamanho máximo, preencha com zeros
            while len(valor) < tamanho_maximo:
                valor.append(0)

    return dic

# generating sheets with result time by id execution
def generatingSheetResultTime(id_exec):
    df = pd.DataFrame()
    df.to_csv('resultTime-'+id_exec+'.csv', header=True, sep=';', mode='a', index=False, encoding='utf-8-sig')

    dados_dict = {}

    with open("./output/results/execution-"+id_exec+"/time.txt") as infile:
        for actual_line in infile:
            partes = actual_line.strip().split(';')
            if len(partes) == 2:
                coluna, valor = partes[0].strip(), partes[1].strip().replace("s","")
                if coluna in dados_dict:
                    dados_dict[coluna].append(valor)
                else:
                    dados_dict[coluna] = [valor]

    aux_dic = adjusting_dict(dados_dict)

    final_dic = normalize_dict(aux_dic)
    
    #save the last group
    df = pd.DataFrame(final_dic)
    df.to_csv('resultTime-'+id_exec+'.csv', index=False)

# generating a CSV file of times for the Nth execution
for i in range(n):
    generatingSheetResultTime(str(i + 1))

In [165]:
#!/usr/bin/python
# generate a csv file with the messages from the conflicts found by scenario and analysis

import pandas as pd
import sys

# Check if n was passed as a parameter, otherwise it will default to 10
n = 5
if len(sys.argv) > 1:
    n = int(sys.argv[1])
else:
    n = 10


def menor_tamanho_chave(dicionario):
    menor_chave = None
    menor_tamanho = float('inf')  # Inicialmente, consideramos o tamanho infinito

    for chave, valor in dicionario.items():
        if isinstance(valor, list) and len(valor) < menor_tamanho:
            menor_chave = chave
            menor_tamanho = len(valor)

    return menor_tamanho
def chaves_com_tamanho_n(dicionario, tamanho):
    chaves = []
    for chave, valor in dicionario.items():
        if isinstance(valor, list) and len(valor) > tamanho:
            chaves.append(chave)
    return chaves

def adjusting_dict(dados_dict):
    # Chave que você deseja dividir
    dicionario = dados_dict

    len_dic = menor_tamanho_chave(dados_dict)

    chaves = chaves_com_tamanho_n(dados_dict, len_dic)

    for chave in chaves:
        # Verifica se a chave existe no dicionário
        if chave in dicionario:
            # Obtém a lista de valores correspondente à chave
            lista_valores = dicionario[chave]

            # Verifica se a lista tem mais de len_dic elementos
            if len(lista_valores) > len_dic:

                # Divide a lista em duas partes
                primeira_parte = lista_valores[::2]
                segunda_parte = lista_valores[1::2]

                # Cria duas novas chaves no dicionário com as partes divididas
                dicionario[chave + ' left-right'] = primeira_parte
                dicionario[chave + ' right-left'] = segunda_parte

                # Exclui a chave antiga
                del dicionario[chave]
            else:
                dicionario[chave] = lista_valores


    # Exibe o dicionário atualizado
    return dicionario


def normalize_dict(dic):
    # Encontre o tamanho máximo entre todas as listas
    tamanho_maximo = max(len(valor) for valor in dic.values())

    # Percorra as chaves do dicionário
    for chave, valor in dic.items():
        if isinstance(valor, list):
            # Se a lista for menor que o tamanho máximo, preencha com zeros
            while len(valor) < tamanho_maximo:
                valor.append(0)

    return dic

# generating sheets with result time by id execution
def generatingSheetResultTime(id_exec):
    df = pd.DataFrame()
    df.to_csv('conflicts_log-'+id_exec+'.csv', header=True, sep=';', mode='a', index=False, encoding='utf-8-sig')
    
    dados_dict = {}
    
    with open("./output/results/execution-"+id_exec+"/conflicts_log.txt") as infile:
        aux_list = ["0" for i in range(6)]
    
        for actual_line in infile:
            
            partes = actual_line.strip().split(': [')
           
            if len(partes) == 2:
                coluna, valor = partes[0].strip(), partes[1].strip()
                
                if coluna in dados_dict:
                    dados_dict[coluna].append(valor)
                else:
                    dados_dict[coluna] = [valor]

    
    aux_dic = adjusting_dict(dados_dict)

    final_dic = normalize_dict(aux_dic)
    
    #save the last group
    df = pd.DataFrame(final_dic)
    df.to_csv('conflicts_log-'+id_exec+'.csv', index=False)

# generating a CSV file of conflicts for the Nth execution
for i in range(n):
    generatingSheetResultTime(str(i + 1))